In [ ]:
!pip install bs4 html2text

  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=7ebb235e4a8eebda8518a1a6a2b5b94304551e49b2ec8f92c3e3c0886c238f45
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
headers = {
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
  'Accept-Language': 'en-US,en;q=0.9',
}

In [ ]:
def get_links_from_page(index):
  # Send a GET request to the webpage you want to crawl
  url = f"https://www.webmd.com/diet/medical-reference/default.htm?pg={index}"
  response = requests.get(url, headers=headers)

  # Parse the HTML content using BeautifulSoup
  soup = BeautifulSoup(response.content, 'html.parser')

  # Find the ul element with class 'az-index-results-group-list'
  ul_element = soup.find('section', class_='dynamic-index-feed').find('ul', class_='list')

  # Find all the links (a tags) within the ul element
  anchors = ul_element.find_all('a')

  # Extract the href attribute from each link
  links = [anchor['href'] for anchor in anchors]

  return links

In [ ]:
"""
get_all_links_from_reference
"""

all_links = []
for index in range(1, 24):
  all_links.extend(get_links_from_page(index))

all_links

['https://www.webmd.com/diet/health-benefits-loquat',
 'https://www.webmd.com/diet/deficiency-symptoms-causes-vitamin-b12',
 'https://www.webmd.com/diet/manage-potassium-dining-out',
 'https://www.webmd.com/diet/guidelines-potassium-intake',
 'https://www.webmd.com/diet/popular-diets-lower-potassium',
 'https://www.webmd.com/diet/follow-low-potassium-diet',
 'https://www.webmd.com/diet/health-benefits-of-wine-cap-mushrooms',
 'https://www.webmd.com/diet/food-deserts-what-know',
 'https://www.webmd.com/diet/what-is-culinary-medicine',
 'https://www.webmd.com/diet/what-to-know-about-sea-vegetables',
 'https://www.webmd.com/diet/health-benefits-of-lobster-mushroom',
 'https://www.webmd.com/diet/health-benefits-cheddar-cheese',
 'https://www.webmd.com/diet/what-to-know-about-ramen-noodles',
 'https://www.webmd.com/diet/what-to-know-about-absinthe',
 'https://www.webmd.com/diet/what-to-know-about-banana-spiders',
 'https://www.webmd.com/diet/what-to-know-dietitians-vs-nutritionists',
 'http

In [ ]:
class Document:
  def __init__(self, title, url, content):
    self.title = title
    self.url = url
    self.content = content

In [ ]:
def scrape_webpage(url):
  response = requests.get(url, headers=headers)
  soup = BeautifulSoup(response.content, 'html.parser')
  return soup

In [ ]:
from html2text import html2text


def webpage_to_markdown(soup):
  # Find the article body, either by `class_='article__body'` or `class_='article-body'`
  article_body = soup.find(class_='article__body')
  if article_body is None:
    article_body = soup.find(class_='article-body')

  # Extract the HTML content from the article body
  html_content = str(article_body)

  # Convert the HTML content into markdown
  markdown_content = html2text(html_content, bodywidth=0)
  return markdown_content

In [ ]:
import re


def strip_links_from_markdown(markdown_content):
  # Make a RegEx pattern
  link_pattern = re.compile(r'\[(.*?)\]\((.*?)\)')

  # Find all matches in markdown_content
  matches = link_pattern.findall(markdown_content)

  for _match in matches:
    full_match = f'[{_match[0]}]({_match[1]})'

    # Replace the full link with just the text
    markdown_content = markdown_content.replace(full_match, _match[0])

  return markdown_content

In [ ]:
import re


def split_markdown_by_headings(markdown_content):
  # Define the regular expression pattern to match H1 and H2 headings
  pattern = r'(#{1,2}.*)'

  # Split the Markdown text based on the headings
  sections = re.split(pattern, markdown_content)

  # Combine each heading with its corresponding text
  combined_sections = []
  for i in range(1, len(sections), 2):
      heading = sections[i].strip()  # Get the heading from sections[i]
      text = sections[i + 1].strip() if i + 1 < len(
          sections) else ''  # Get the text from sections[i + 1] if it exists, otherwise use an empty string
      combined_section = f"{heading}\n{text}"  # Combine the heading and text using a newline character
      combined_sections.append(combined_section)  # Add the combined section to the list

  if len(combined_sections) == 0:
    combined_sections = [markdown_content]

  return combined_sections

In [ ]:
def create_documents_from_webpage(url):
  # Wrap function in a try-except block to handle any potential exceptions
  try:
    soup = scrape_webpage(url)

    # Extract the title from the article
    title = soup.find('h1').get_text().strip()

    # Convert the webpage HTML to markdown
    markdown_content = webpage_to_markdown(soup)

    # Strip links from the markdown content
    markdown_content = strip_links_from_markdown(markdown_content)

    # Split the article up headings
    contents_by_heading = split_markdown_by_headings(markdown_content)
    docs = []
    for content in contents_by_heading:
      docs.append(Document(title=title, url=url, content=content))
    return docs
  except:
    return []

In [ ]:
"""
generate_databank
"""

docs = []

# Iterate through every link in the Reference
for link in all_links:
  print(f"{all_links.index(link)+1}/{len(all_links)}")
  print(f"{link}")

  # Add all of the documents created from the article
  docs_from_article = create_documents_from_webpage(link)
  docs.extend(docs_from_article)

  print(f"Processed {len(docs_from_article)} documents.\n")

1/1143
https://www.webmd.com/diet/health-benefits-loquat
Processed 3 documents.

2/1143
https://www.webmd.com/diet/deficiency-symptoms-causes-vitamin-b12
Processed 8 documents.

3/1143
https://www.webmd.com/diet/manage-potassium-dining-out
Processed 10 documents.

4/1143
https://www.webmd.com/diet/guidelines-potassium-intake
Processed 6 documents.

5/1143
https://www.webmd.com/diet/popular-diets-lower-potassium
Processed 7 documents.

6/1143
https://www.webmd.com/diet/follow-low-potassium-diet
Processed 5 documents.

7/1143
https://www.webmd.com/diet/health-benefits-of-wine-cap-mushrooms
Processed 8 documents.

8/1143
https://www.webmd.com/diet/food-deserts-what-know
Processed 4 documents.

9/1143
https://www.webmd.com/diet/what-is-culinary-medicine
Processed 5 documents.

10/1143
https://www.webmd.com/diet/what-to-know-about-sea-vegetables
Processed 6 documents.

11/1143
https://www.webmd.com/diet/health-benefits-of-lobster-mushroom
Processed 6 documents.

12/1143
https://www.webmd.co

In [ ]:
import pickle

with open('docs.pkl', "wb") as file:
  pickle.dump(docs, file)
  file.close()

Please run the above code cells and download `docs.pkl` to use in the next tutorials!